<img src="logo_LNEC.png" alt="LNEC" width="200" align="right"/>

# ERIES - RE-SAFE PROJECT - Claudio Mazzotti (_Università di Bologna_)

## Specimen for the shake table test

The RC frame specimen is built at a scale of 2/3 using the Cauchy-Froude similitude law.

## Shake table input motions

Two different earthquake records are under consideration for the shake table test at LNEC:

1. The 2009 L'Aquila earthquake (L'AQUILA-IT.AVZ.00.HNN.D.IT-2009)

0. The 1976 Friuli earthquake (TOLMEZZO)

## Computational environment

In [26]:
# IPython magic commands
%matplotlib inline

# Developed for Python 3.12.4
from __future__ import annotations

# Python standard library
import os
import sys
import datetime

from pathlib import Path

# 3rd party modules
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib as mpl

from scipy import constants
from scipy.signal import butter, sosfiltfilt, periodogram, resample
from scipy.signal.windows import tukey

import matplotlib.pyplot as plt

# pxc modules
if (ppfolder := str(Path('..') / 'personal-packages3')) not in sys.path:
    sys.path.append(ppfolder)


import utilities
import LNECSPA
from utilities import maxabs
from LNECSPA import LTFdb

# missing imports
""" import PPcore
import RSnumba
import DSP
import LNEC3DST
from RSnumba import cts as RScalc
from DSP import FrequencyDomain, dBpow

# print environment
PPcore.environment(globals()) """

' import PPcore\nimport RSnumba\nimport DSP\nimport LNEC3DST\nfrom RSnumba import cts as RScalc\nfrom DSP import FrequencyDomain, dBpow\n\n# print environment\nPPcore.environment(globals()) '

In [27]:
# Don't run this one
if False:
    ttA2 = dfA2.index
    dtA2 = ttA2[1] - ttA2[0]
    SpoA2 = dfA2['Spo'].to_numpy()

    sos = butter(8, 40.*np.sqrt(SF), output='sos', fs=1./dtA2)
    SpoA3 = sosfiltfilt(sos, SpoA2, padtype='even')

    SpoA3 *= tukey(SpoA3.size, alpha=0.2) # 20% Tukey window

    SpoA3, ttA3 = resample(SpoA3, int(ttA2.size*np.sqrt(SF)), ttA2)
    dtA3 = ttA3[1] - ttA3[0]
    print(dtA3) # before rounding

    dtA3 = np.round(dtA3, decimals=3)
    ttA3 = dtA3 * np.arange(ttA3.size)
    print(ttA2[-1], ttA3[-1], dtA3, ttA3.size) # after rounding

    VelA3 = FrequencyDomain.differentiate(SpoA3, dt=dtA3)
    AccA3 = FrequencyDomain.differentiate(VelA3, dt=dtA3)

    fig, ax = plt.subplots(nrows=3, sharex=True, figsize=(18,6), layout='constrained')
    ax[-1].set_xlabel('Time (s)')

    for axis, column, ts, units in zip(ax, ('Spo', 'Vel', 'Acc'), (SpoA3, VelA3, AccA3), ('cm', 'cm/s', 'cm/s²')):
        axis.plot(dfA2[column], label=f'Scalato (peak={maxabs(dfA2[column]):.1f}{units})')
        axis.plot(ttA3, ts, label=f'Processed (peak={maxabs(ts):.1f}{units})')
        axis.set_ylabel(f'{column} ({units})')
        axis.legend(loc='upper right')
        axis.grid(visible=True)

    ## Export data
    fA3 = pd.DataFrame(data=(SpoA3, VelA3, AccA3), index=('Spo (cm)', 'Vel (cm/s)', 'Acc (cm/s²)'), columns=ttA3).T
    dfA3.index.name = 'Time (s)'
    dfA3RS = pd.DataFrame(data=rsa, index=TT, columns=('Acc (cm/s²)',))
    dfA3RS.index.name = 'Time (s)'

    with pd.ExcelWriter("L'AQUILA-IT.AVZ.00.HNN.D.IT-2009_LNEC3D.xlsx") as writer:
        dfA3.to_excel(writer, sheet_name='Time series', index=True)
        dfA3RS.to_excel(writer, sheet_name='Response spectrum', index=True)

    # LTF file
    ltfA = LTFdb()
    ltfA.update(AQTD=['AQTD'],
                stage=["L'AQUILA-IT.AVZ.00.HNN.D.IT-2009 seismic record, scale factor 2/3, Cauchy+Froude similitude law"],
                obs=['ERIES RE-Safe project'],
                date=[str(datetime.datetime.now())],
                t0=np.repeat(ltfA.t0, 3),
                dt=np.repeat(ltfA.dt*0.005, 3),
                dataformat=np.repeat(ltfA.dataformat, 3),
                IDstring=ltfA.IDstring*3,
                scalefactor=np.repeat(ltfA.scalefactor, 3),
                offset=np.repeat(ltfA.offset, 3),
                data=np.vstack((SpoA3*10., VelA3, AccA3/100./constants.g)), # acceleration is in G's
                names=['DispT', 'VelT', 'AccT'],
                units=['mm', 'cm/s', 'g'],  # attention to units (Not SI)
                types=['Displacement', 'Velocity', 'Acceleration'],
                info=[f'PGD={maxabs(SpoA3*10.):.0f}mm', f'PGV={maxabs(VelA3):.2f}cm/s', f'PGA={maxabs(AccA3/100./constants.g):.3f}g'],
                )
    ltfA.write('LAquilaReducedScale.ltf')

    # LNEC3D shake table target motion file
    tgtA = LTFdb()
    tgtA.update(AQTD=['AQTD'],
                stage=["L'AQUILA-IT.AVZ.00.HNN.D.IT-2009 seismic record, scale factor 2/3, Cauchy+Froude similitude law"],
                obs=['ERIES RE-Safe project'],
                date=[str(datetime.datetime.now())],
                t0=np.repeat(tgtA.t0, 6),
                dt=np.repeat(tgtA.dt*0.005, 6),
                dataformat=np.repeat(tgtA.dataformat, 6),
                IDstring=tgtA.IDstring*6,
                scalefactor=np.repeat(tgtA.scalefactor, 6),
                offset=np.repeat(tgtA.offset, 6),
                data=np.vstack((SpoA3*10., SpoA3*0., SpoA3*0., AccA3/100./constants.g, AccA3*0., AccA3*0.)),
                names=['DispT', 'DispL', 'DispV', 'AccT', 'AccL', 'AccV'],
                units=['mm']*3 + ['g']*3,
                types=['Displacement']*3 + ['Acceleration']*3,
                info=tgtA.info*6,
                )
    tgtA.write('LAquilaReducedScale.tgt')

In [29]:
tgtA = LTFdb()
tgtA.read(r'C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\LNEC_Adapta_Driver\2024_ERIES_RE-Safe\Targets\LAquilaReducedScale.tgt')
print(tgtA)
print()
#tgtA.print_metadata()


Filename: C:\Users\afons\OneDrive - Universidade de Lisboa\Controlo de Plataforma Sismica\LNEC_Adapta_Driver\2024_ERIES_RE-Safe\Targets\LAquilaReducedScale.tgt
AQTD: ['AQTD']
Stage: ["L'AQUILA-IT.AVZ.00.HNN.D.IT-2009 seismic record, scale factor 2/3, Cauchy+Froude similitude law"]
Obs: ['ERIES RE-Safe project']
Date: ['2024-07-19 12:18:26.272293']
Signals: 6



In [ ]:
# AI generated code for file format conversion to .txt and to SI units

import numpy as np
from scipy.constants import g

# Assuming tgtA is already loaded via LTFdb.read

# 1. Identify channel indices
disp_inds = [i for i, unit in enumerate(tgtA.units) if unit.lower() in ('mm', 'm')]
acc_inds  = [i for i, typ  in enumerate(tgtA.types) if typ.lower() == 'acceleration']

# 2. Build time vector
dt     = float(tgtA.dt[0])
n_pts  = tgtA.data.shape[1]
time_s = np.arange(n_pts) * dt

# 3. Convert to SI
# Displacement (to meters)
disp_si = tgtA.data[disp_inds].astype(float)
for idx, ch in enumerate(disp_inds):
    if tgtA.units[ch].lower() == 'mm':
        disp_si[idx] *= 1e-3

# Acceleration (to m/s^2)
acc_si = tgtA.data[acc_inds].astype(float)
for idx, ch in enumerate(acc_inds):
    if tgtA.units[ch].lower() == 'g':
        acc_si[idx] *= g

# 4. Stack columns: time | displacements | accelerations
out_arr = np.vstack((time_s, disp_si, acc_si)).T  # shape (n_pts, 1 + n_disp + n_acc)

# 5. Remove columns that are entirely zero
mask = ~np.all(np.isclose(out_arr, 0.0), axis=0)
out_arr = out_arr[:, mask]

# 6. Prepare headers aligned with filtered columns
header_cols = ['time_s'] + \
    [f"{tgtA.names[i]}_m" for i in disp_inds] + \
    [f"{tgtA.names[i]}_m_per_s2" for i in acc_inds]
header_cols = [h for h, m in zip(header_cols, mask) if m]

# 7. Export with fixed-width columns and aligned headers
width = 15
fmt = ['%15.6e'] * len(header_cols)

# Build header line with each column name centered
header_line = ''.join(h.center(width) for h in header_cols)

txt_filename = 'tgtA_SI_time_disp_acc.txt'
with open(txt_filename, 'w') as f:
    f.write(header_line + '\n')
    np.savetxt(f, out_arr, fmt=fmt)

print(f"Exported {out_arr.shape[0]} samples and {out_arr.shape[1]} columns to '{txt_filename}'.")


Exported 8982 samples and 3 columns to 'tgtA_SI_time_disp_acc.txt'.
